# sklearn IRIS
![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=555&t=event&ec=benchmark&ea=bentoml-benchmark&dt=sklearn-iris-clipper)

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline


In [5]:
import bentoml
NAME = 'sklearn_iris_clipper'

# load data & train model

In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import datasets

iris = datasets.load_iris()
x = iris.data[:, 2:]
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=7)

    
# add parameters for tuning
num_estimators = 100

# train the model
model = RandomForestRegressor(n_estimators=num_estimators)
model.fit(X_train, y_train)
predictions = model.predict(X_test)
print('predictions: ', predictions)

# log model performance 
mse = mean_squared_error(y_test, predictions)
print("  mse: %f" % mse)

predictions:  [2.   1.   0.   1.   1.62 0.   1.23 1.   0.   1.   1.99 1.   0.   2.
 0.   1.84 2.   2.   0.   0.   1.   2.   1.   1.36 1.51 1.84 1.   1.
 2.   2.  ]
  mse: 0.085277


In [7]:
%%writefile {NAME}.py

import bentoml
from bentoml.artifact import SklearnModelArtifact
from bentoml.handlers import ClipperFloatsHandler


@bentoml.env()
@bentoml.artifacts([SklearnModelArtifact('model')])
class BentoSvc(bentoml.BentoService):

    @bentoml.api(ClipperFloatsHandler)
    def predict_clipper(self, inputs):
        outputs = self.artifacts.model.predict(inputs)
        return outputs

Writing sklearn_iris_clipper.py


In [8]:
from sklearn_iris_clipper import BentoSvc

bento_svc = BentoSvc()
bento_svc.pack("model", model)
saved_path = bento_svc.save()

[2020-05-11 03:46:47,506] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.4.9. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-05-11 03:46:47,848] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.4.9. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml

no previously-included directories found matching 'e2e_tests'
no previously-included directories found matching 'tests'
no previously-included directories found matching 'benchmark'


writing manifest file 'BentoML.egg-info/SOURCES.txt'
running check
creating BentoML-0.4.9+220.gccdf8b4.dirty
creating BentoML-0.4.9+220.gccdf8b4.dirty/BentoML.egg-info
creating BentoML-0.4.9+220.gccdf8b4.dirty/bentoml
creating BentoML-0.4.9+220.gccdf8b4.dirty/bentoml/artifact
creating BentoML-0.4.9+220.gccdf8b4.dirty/bentoml/bundler
creating BentoML-0.4.9+220.gccdf8b4.dirty/bentoml/cli
creating BentoML-0.4.9+220.gccdf8b4.dirty/bentoml/clipper
creating BentoML-0.4.9+220.gccdf8b4.dirty/bentoml/configuration
creating BentoML-0.4.9+220.gccdf8b4.dirty/bentoml/configuration/__pycache__
creating BentoML-0.4.9+220.gccdf8b4.dirty/bentoml/deployment
creating BentoML-0.4.9+220.gccdf8b4.dirty/bentoml/deployment/aws_lambda
creating BentoML-0.4.9+220.gccdf8b4.dirty/bentoml/deployment/sagemaker
creating BentoML-0.4.9+220.gccdf8b4.dirty/bentoml/handlers
creating BentoML-0.4.9+220.gccdf8b4.dirty/bentoml/marshal
creating BentoML-0.4.9+220.gccdf8b4.dirty/bentoml/migrations
creating BentoML-0.4.9+220.gccd

# Build & Run Bento Service in Docker

In [1]:
from clipper_admin import ClipperConnection, DockerContainerManager
from bentoml.clipper import deploy_bentoml
cl = ClipperConnection(DockerContainerManager())
try:
    cl.start_clipper(cache_size=1)  # limit cache size for comparison
except:
    cl.connect()

20-05-11:04:51:14 INFO     [__init__.py:118] No local BentoML config file found, using default configurations
20-05-11:04:51:15 INFO     [docker_container_manager.py:184] [default-cluster] Starting managed Redis instance in Docker
20-05-11:04:51:18 INFO     [docker_container_manager.py:276] [default-cluster] Metric Configuration Saved at /tmp/tmpadvh48aw.yml
20-05-11:04:51:19 INFO     [clipper_admin.py:162] [default-cluster] Clipper is running


In [3]:
APP_NAME = saved_path.split('/')[-1].lower()
cl.register_application(APP_NAME, 'floats', 'default_pred', 300000)

20-05-11:04:51:51 INFO     [clipper_admin.py:236] [default-cluster] Application 20200511034647_f301a8 was successfully registered


In [4]:
clipper_model_name, clipper_model_version = deploy_bentoml(
    cl, saved_path, 'predict_clipper',
    build_envs=dict()
)

[2020-05-11 04:51:59,246] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.4.9. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-05-11 04:51:59,262] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.4.9,  but loading from BentoML version 0.4.9+220.gccdf8b4.dirty
[2020-05-11 04:51:59,905] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image

20-05-11:04:52:00 INFO     [docker_container_manager.py:409] [default-cluster] Found 0 replicas for bentosvc-predict-clipper:20200511034647-f301a8. Adding 1
20-05-11:04:52:01 INFO     [clipper_admin.py:724] [default-cluster] Successfully registered model bentosvc-predict-clipper:20200511034647-f301a8
20-05-11:04:52:01 INFO     [clipper_admin.py:642] [default-cluster] Done deploying model bentosvc-predict-clipper:20200511034647-f301a8.


[2020-05-11 04:52:01,131] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.4.9. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-05-11 04:52:01,142] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.4.9,  but loading from BentoML version 0.4.9+220.gccdf8b4.dirty


In [5]:
cl.set_num_replicas(clipper_model_name, 1)  # limit workers for comparison
cl.link_model_to_app(APP_NAME, clipper_model_name)
addr = cl.get_query_addr()
server_url = f"http://{addr}/{APP_NAME}/predict"

20-05-11:04:52:18 INFO     [clipper_admin.py:303] [default-cluster] Model bentosvc-predict-clipper is now linked to application 20200511034647_f301a8


In [6]:
server_url

'http://localhost:1337/20200511034647_f301a8/predict'

# Test with requests

In [7]:
import json
import requests

from sklearn import datasets
from sklearn.model_selection import train_test_split

iris = datasets.load_iris()
x = iris.data[:, 2:]
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=7)

headers = {"content-type": "application/json"}
data = json.dumps(
       {"input": X_test[0].tolist()}
)

json_response = requests.post(server_url, data=data, headers=headers)
print(json_response)
print(json_response.text)

<Response [200]>
{"query_id":0,"output":2.0,"default":false}


# Benchmark

In [8]:
import pandas as pd
import json
import copy
import random


def get_request_producer():

    from sklearn import datasets
    from sklearn.model_selection import train_test_split

    iris = datasets.load_iris()
    x = iris.data[:, 2:]
    y = iris.target

    url = server_url
    method = "POST"
    headers = {"content-type": "application/json"}
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=7)
    base_data = X_test[0].tolist()

    def _gen_data():
        raw = copy.deepcopy(base_data)
        raw[0] += random.random() / 10000
        raw[1] += random.random() / 10000
        data = json.dumps({"input": raw})
        return url, method, headers, data

    return _gen_data

get_request_producer()()

('http://localhost:1337/20200511034647_f301a8/predict',
 'POST',
 {'content-type': 'application/json'},
 '{"input": [5.100028533214343, 1.800057818481174]}')

In [9]:
# When it reached the limit of clipper, clipper will response with the default value
def verify_clipper_response(status, msg):
    if status // 100 == 2 and "default_pred" not in msg:
        return True
    return False

In [14]:
from bentoml.utils.benchmark import BenchmarkClient
b = BenchmarkClient(get_request_producer(), lambda: 1, timeout=0.32,
                    verify_response=verify_clipper_response)
b.start_session(60, 900, 600)

======= Session started! =======

╒══════════╤═════════╤══════════╤═════════════════╤═══════════════════╕
│ Result   │   Total │   Reqs/s │   Resp Time Avg │ Client Health %   │
╞══════════╪═════════╪══════════╪═════════════════╪═══════════════════╡
│ succ     │       0 │        0 │             nan │ 100.0             │
├──────────┼─────────┼──────────┼─────────────────┼───────────────────┤
│ fail     │       0 │        0 │             nan │                   │
╘══════════╧═════════╧══════════╧═════════════════╧═══════════════════╛

╒══════════╤═════════╤══════════╤═════════════════╤═══════════════════╕
│ Result   │   Total │   Reqs/s │   Resp Time Avg │ Client Health %   │
╞══════════╪═════════╪══════════╪═════════════════╪═══════════════════╡
│ succ     │     613 │      296 │        0.11002  │ 100.0             │
├──────────┼─────────┼──────────┼─────────────────┼───────────────────┤
│ fail     │     162 │       81 │        0.342643 │                   │
╘══════════╧═════════╧═══════